In [ ]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd             # Для удобной загрузки данных в БД              # Для подключения к СУБД
import datetime
import numpy as np

def get_page(text, pg=0):
    # Справочник параметров GET-запроса
    params = {
        'text': text,               # Текст поискового запроса
        'area': 1,                  # Индекс города Москва
        'page': pg,                 # Индекс страницы поиска на HH
        'per_page': 100             # Кол-во вакансий на 1 странице
    }
    # Чтобы спарсить вакансии со страницы, нужно выполнить методом GET http-запрос,
    # включая параметры фильтра поиска (ответ приходит в формате json):
    url = 'https://api.hh.ru/vacancies'
    
    req = requests.get(url, params, timeout=20)  # Запрос к API
    data = req.content.decode()                 # Декодирование ответа (для Кириллицы)
    return data

text_filter = '"Data Engineer" OR "ML Engineer" OR "ETL Developer"'

page_result_list = []
# Считывание 10000 вакансий (100 страниц по 100 вакансий)
for page in range(0, 100):
    print(page)
    # Преобразование текстового ответа запроса в словарь
    page_dict = json.loads(get_page(text_filter, page))

    # Сохранение страниц в список
    page_result_list+=page_dict['items']

    # Условие выхода из цикла (если страниц окажется меньше 100)
    if (page_dict['pages'] - page) <= 1:
        break
    # Задержка, чтобы не нагружать сервисы hh
    time.sleep(0.25)
print('Страницы поиска собраны')

repr= []
for i in page_result_list:
    try:
        d_mvf={'id':str(i['id']),
        'name':i['name'],
        'department':i['department'] if i['department']==None else i['department']['name'],
        'response_letter_required':1 if i['response_letter_required'] else 0,
        'type':i['type']['name'],
        'lat':i['address']['lat'],
        'lng':i['address']['lng'],
        'created_at':i['created_at'][:-6],
        'employer': i['employer']['name'],
        'accredited_it_employer': 1 if 'accredited_it_employer' in i['employer'].keys() else 0,
        'schedule':i['schedule']['name'],
        'accept_temporary':1 if i['accept_temporary'] else 0,
        'professional_roles':i['professional_roles'][0]['name'],
        'experience':np.mean([int(i) for i in i['experience']['name'].split(' ') if i.isnumeric()]),
        'employment':i['employment']['name']}
        
        if 'salary' in i.keys():
            d_mvf['salary']=i['salary'] if i['salary']==None else i['salary']['from']
        repr+=[d_mvf]
    except:
        pass
repr = pd.DataFrame(repr)
repr[['response_letter_required','accredited_it_employer','accept_temporary']]=repr[['response_letter_required','accredited_it_employer','accept_temporary']].fillna(0).astype(str)

import sqlite3

# Устанавливаем соединение с базой данных
connection = sqlite3.connect('my_database.db')
cursor = connection.cursor()


# Выбираем всех пользователей
cursor.execute('SELECT id FROM resume_data ')
users = cursor.fetchall()

users = [i[0] for i in list(users)]
print(list(users))

repr= repr.loc[~repr.id.isin(list(users))]

for i in range(len(repr)):
    data= list(repr.iloc[i])
    print(data)
    cursor.execute('''INSERT INTO resume_data (
    id,name,department,response_letter_required,type,lat,lng,created_at,employer,accredited_it_employer,schedule,accept_temporary,professional_roles,experience,employment,salary

    ) VALUES (?, ?, ?, ?, ?,?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?)''', data)
    connection.commit()
    
connection.close()

In [ ]:
import sqlite3

# Устанавливаем соединение с базой данных
connection = sqlite3.connect(r'C:\Users\nasty\OneDrive\Рабочий стол\hh')
cursor = connection.cursor()


# Выбираем всех пользователей
cursor.execute('SELECT * FROM resume_data ')
users = cursor.fetchall()
df = pd.DataFrame(list(users),columns = ['id', 'name', 'department', 'response_letter_required', 'type', 'lat',
    'lng', 'created_at', 'employer', 'accredited_it_employer', 'schedule',
    'accept_temporary', 'professional_roles', 'experience', 'employment',
    'salary'])

df['date'] = pd.to_datetime(df['created_at'])
df[['response_letter_required','accredited_it_employer','accept_temporary',]]

In [ ]:
import sqlite3
# Устанавливаем соединение с базой данных

dir_current = os.getcwd()
# Устанавливаем соединение с базой данных
connection = sqlite3.connect(dir_current+'/my_database.db')
cursor = connection.cursor()

cursor.execute('''drop TABLE resume_data ''')
# Создаем таблицу Users
cursor.execute('''CREATE TABLE IF NOT EXISTS resume_data (
id                           text,
name                         text,
department                   text,
response_letter_required       text,
type                         text,
lat                         float,
lng                         float,
created_at                   date,
employer                     text,
accredited_it_employer         text,
schedule                     text,
accept_temporary               text,
professional_roles           text,
experience                  float,
employment                   text,
salary                      float

)''')

# Сохраняем изменения и закрываем соединение
connection.commit()
connection.close()